In [1]:
import cv2
import time
import mediapipe as mp

In [33]:
video = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
drawing = mp.solutions.drawing_utils

tipIds = [4, 8, 12, 16, 20] # Parmak uç noktaları id'leri

while True:
    retval, img = video.read()
    if not retval: break
    
    img = cv2.flip(src=img, flipCode=1)
    results = hands.process(image=img).multi_hand_landmarks
    
    landMarkList = []
    if results:
        for handlandmark in results:
            drawing.draw_landmarks(image=img,
                                   landmark_list=handlandmark,
                                   connections=mpHands.HAND_CONNECTIONS)
            
            for id, landmark in enumerate(handlandmark.landmark):
                height, width, color = img.shape
                cx, cy = int(landmark.x * width), int(landmark.y * height)
                landMarkList.append([id, cx, cy])
               
    
    if len(landMarkList) != 0:
        fingers = []
        
        # baş parmak
        if landMarkList[tipIds[0]][1] < landMarkList[tipIds[0] - 1][1]:
            fingers.append(1)
        else: 
            fingers.append(0)
        
        # 4 parmak
        for id in range(1, 5):
            
            if landMarkList[tipIds[id]][2] < landMarkList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
       
    
        totalF = fingers.count(1)
        cv2.putText(img=img, 
                    org=(500, 30),
                    text=f"{totalF}",
                    color=(0, 0, 255),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    thickness=2)
    
    
    
    
    
    cv2.imshow(winname='Video', mat=img)
    
    if cv2.waitKey(1) == ord('q'): break

video.release()
cv2.destroyAllWindows() 